# Work in Progress to plot results

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# Plot benchmarks using existing run data

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import sys


#CSV_PATH = sys.argv[1]
CSV_PATH = "results/2021-12-13-14:14:51"

header = ['version', 'workload', 'runs', 'warmup', 'blocks', 'threads_per_block', 'num_floats', 'malloc_mean', 'malloc_max', 'free_mean', 'free_max', 'work_mean', 'work_max']
df = pd.read_csv(CSV_PATH+"/out.csv", names = header, header = 0, delimiter = ' ')


# single plot
def plot_floats_vs_cycles(df, ax, params):
        
    # filter for data that fits the given parameters
    block_mask = df['blocks'] == params['blocks']
    threads_mask = df['threads_per_block'] == params['threads_per_block']
    workload_mask = df['workload'] == params['workload']
    df = df[block_mask & threads_mask & workload_mask]
    
    if params['num_floats'] >= 0:
        float_mask = df['num_floats'] <= params['num_floats']
        df = df[float_mask]
    
    
    # calculate mean and std over all runs
    df = df.groupby(['num_floats', 'version'], as_index=False)\
           .agg({'malloc_mean':['mean','std'], 'malloc_max':['mean', 'std'],\
                 'free_mean':['mean','std'],   'free_max':['mean', 'std'],\
                 'work_mean':['mean','std'],   'work_max':['mean', 'std']}\
               )
    
   
    floats = list(df['num_floats'].unique())
    
    # select versions to plot
    if len(params['versions']) != 0:
        versions = params['versions']
    else:
        versions = list(df['version'].unique())
    
    # plot lines
    measurement_type = params['measurement_type'] # is either "malloc", "work", or "free"
    aggregation_type = params['aggregation_type'] # is either "mean" or "max"
    ax.set_title(measurement_type)
    # baseline reference
    reference_mean = df[df['version'] == versions[0]][measurement_type + '_' + aggregation_type]['mean']
    
    for version in versions:
        mean = df[df['version'] == version][measurement_type + '_' + aggregation_type]['mean']
        std = df[df['version'] == version][measurement_type + '_' + aggregation_type]['std']
        
        # relative speedup to first version (baseline)
        if params['show_speedup']:
            speedup = np.asarray(reference_mean) / np.asarray(mean)
            ax.plot(floats, speedup, label=str(version))
            ax.set_ylabel("relative speedup")
        else:
            ax.errorbar(floats, mean, std, label=str(version))
            ax.set_ylabel("#clock cycles")
    
    ax.legend()
    ax.set_xlabel("#floats")
    return
    

    
# extract range of parameters
versions = list(df['version'].unique())
workloads = list(df['workload'].unique())
runs = list(df['runs'].unique())
warmups = list(df['warmup'].unique())
blocks = list(df['blocks'].unique())
threads_per_block = list(df['threads_per_block'].unique())
num_floats = list(df['num_floats'].unique())
measurement_types = ['malloc', 'work', 'free']

def plot_workloads_vs_measurement_type():
    ## plot (workloads) vs (malloc, work, free)
    fig, axs = plt.subplots(len(workloads), 3, sharey=False, sharex=True, figsize=(15, len(workloads)*5))
    # constant parameters
    n_blocks = 192
    n_threads_per_block = 64
    n_floats = 10
    versions = [] # all
    aggregation_type = 'mean' # or max
    show_speedup = False
    
    fig.suptitle("Parameters:" \
              + " blocks: " + str(n_blocks)\
              + ", threads per block: " + str(n_threads_per_block)\
              + ", aggregation type: " + str(aggregation_type)\
              + ", runs: " + str(runs)\
              + ", warmup runs: " + str(warmups) + ",\n"\
              + "workloads top-to-bottom: " + str(workloads))
    #fig.supxlabel('Measurement type')
    #fig.supylabel('Workload')

    for w, workload in enumerate(workloads):
        for i, measurement_type in enumerate(measurement_types):
            params = {'measurement_type': measurement_type,
                      'blocks': 192,
                      'threads_per_block': 64,
                      'workload': workload,
                      'num_floats': n_floats,
                      'versions': versions,
                      'aggregation_type': aggregation_type,
                      'show_speedup': show_speedup}
            if len(workloads) == 1:
                ax = axs[i]
            else:
                ax = axs[w][i]
            plot_floats_vs_cycles(df, ax, params)

    plt.savefig(CSV_PATH+"/workload_vs_measurement_type.png", dpi=60, bbox_inches='tight')   
    plt.show()

def plot_blocks_vs_threads_per_block():
    ## plot (blocks) vs (threads_per_block)
    fig, axs = plt.subplots(len(workloads), len(threads_per_block), sharey='row', sharex=True, figsize=(len(threads_per_block)*5, len(workloads)*5))
    # constant parameters
    n_floats = 10
    versions = ['baseline', 'v1_martin', 'v4_anton', 'v5_anton']
    aggregation_type = 'mean' # or max
    show_speedup = True
    
    fig.suptitle("Parameters:" \
              + " aggregation type: " + str(aggregation_type)\
              + ", runs: " + str(runs)\
              + ", warmup runs: " + str(warmups) + ",\n"\
              + "workloads top-to-bottom: " + str(workloads))
    #fig.supxlabel('Measurement type')
    #fig.supylabel('Workload')

    for w, workload in enumerate(workloads):
        for b, block in enumerate(blocks):
        
            params = {'measurement_type': 'work',
                      'blocks': block,
                      'threads_per_block': threads_per_block[b],
                      'workload': workload,
                      'num_floats': n_floats,
                      'versions': versions,
                      'aggregation_type': aggregation_type,
                      'show_speedup': show_speedup}
            if len(workloads) == 1:
                ax = axs[b]
            else:
                ax = axs[w][b]
            plot_floats_vs_cycles(df, ax, params)
            ax.set_title("Work ["+str(block)+", "+str(threads_per_block[b])+"]")

    plt.savefig(CSV_PATH+"/blocks_vs_threads_per_block_vs_workload.png", dpi=60, bbox_inches='tight')   
    plt.show()

    
#plot_workloads_vs_measurement_type()
plot_blocks_vs_threads_per_block()